## 6. VQE로 수소 분자 에너지 

VQE 를 이용해서 분자별 에너지를 계산한 데이터를 추출하였습니다.
- H, LiH, BeH2 기저에너지


### 🙏 참고한 자료
- [qiskit - Quantum Kernel Machine Learning](https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html)

In [12]:
# 수소 분자 에너지 계산
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# 1. 수소 분자 설정
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')
problem = driver.run()
# for label, coeff in sorted(hamiltonian.items()):
#     print(f"{coeff:+.8f} * '{label}'")
    
# 2. Second-quantized Hamiltonian 생성
second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]
# hamiltonian = problem.hamiltonian.second_q_op()

# 3. 매핑: Jordan-Wigner → Qubit 연산자
mapper = JordanWignerMapper()
qubit_op = mapper.map(main_op)#hamiltonian

# 4. Ansatz 및 Optimizer
ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
optimizer = COBYLA(maxiter=100)
estimator = AerEstimator()  # Qiskit Aer 백엔드 사용

# 5. VQE 실행
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

# 6. 결과 출력
print("H₂ 분자의 바닥상태 에너지 (VQE):", result.eigenvalue.real)


H₂ 분자의 바닥상태 에너지 (VQE): -1.8284225221348807


In [13]:
print(second_q_ops)

(FermionicOp({'+_0 -_0': np.float64(-1.25633907300325), '+_1 -_1': np.float64(-0.471896007281142), '+_2 -_2': np.float64(-1.25633907300325), '+_3 -_3': np.float64(-0.471896007281142), '+_0 +_0 -_0 -_0': np.float64(0.3378550774017582), '+_0 +_1 -_1 -_0': np.float64(0.3322908651276483), '+_0 +_2 -_2 -_0': np.float64(0.3378550774017582), '+_0 +_3 -_3 -_0': np.float64(0.3322908651276483), '+_0 +_0 -_1 -_1': np.float64(0.09046559989211571), '+_0 +_1 -_0 -_1': np.float64(0.09046559989211571), '+_0 +_2 -_3 -_1': np.float64(0.09046559989211571), '+_0 +_3 -_2 -_1': np.float64(0.09046559989211571), '+_1 +_0 -_1 -_0': np.float64(0.09046559989211571), '+_1 +_1 -_0 -_0': np.float64(0.09046559989211571), '+_1 +_2 -_3 -_0': np.float64(0.09046559989211571), '+_1 +_3 -_2 -_0': np.float64(0.09046559989211571), '+_1 +_0 -_0 -_1': np.float64(0.3322908651276483), '+_1 +_1 -_1 -_1': np.float64(0.3492868613660083), '+_1 +_2 -_2 -_1': np.float64(0.3322908651276483), '+_1 +_3 -_3 -_1': np.float64(0.34928686136

In [14]:
# H,Li, BeH2 계산
import pandas as pd
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# 분자 리스트
molecules = [
    {"name": "H2", "atom": "H 0 0 0; H 0 0 0.735"},
    {"name": "LiH", "atom": "Li 0 0 0; H 0 0 1.6"},
    {"name": "BeH2", "atom": "Be 0 0 0; H 0 0 1.2; H 0 0 -1.2"},
]

results = []

for mol in molecules:
    try:
        driver = PySCFDriver(atom=mol["atom"], basis="sto3g")
        problem = driver.run()
        # hamiltonian = problem.hamiltonian.second_q_op()
        second_q_ops = problem.second_q_ops()
        main_op = second_q_ops[0]

        mapper = JordanWignerMapper()
        qubit_op = mapper.map(main_op)

        ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
        optimizer = COBYLA(maxiter=100)
        estimator = AerEstimator()

        vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
        result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

        gate_count = ansatz.size()
        circuit_depth = ansatz.depth()

        results.append({
            "molecule": mol["name"],
            "num_qubits": qubit_op.num_qubits,
            "energy": result.eigenvalue.real,
            "gate_count" : gate_count,
            "circuit_depth" : circuit_depth
        })  

        print(f"{mol['name']} 계산 완료: Energy = {result.eigenvalue.real:.6f}")

    except Exception as e:
        print(f"{mol['name']} 계산 중 오류 발생: {e}")
        results.append({
            "molecule": mol["name"],
            "num_qubits": "Error",
            "energy": str(e)
        })

# CSV 저장
df = pd.DataFrame(results)
df.to_csv("data/vqe_molecule_energies.csv", index=False)
print("결과가 vqe_molecule_energies.csv 파일로 저장되었습니다.")


H2 계산 완료: Energy = -1.838282
LiH 계산 완료: Energy = -7.223646
BeH2 계산 완료: Energy = -16.945607
✅ 결과가 vqe_molecule_energies.csv 파일로 저장되었습니다.


In [ ]:
df = pd.read_csv("vqe_molecule_energies.csv")

# 문자열(molecule) → 숫자(one-hot)
df = pd.get_dummies(df, columns=["molecule"])

# 입력값 (X), 출력값 (y) 분리
X = df.drop(columns=["energy"])
y = df["energy"]


## ✍️ 배운 점
- 

## 📌 다음 목표
- 


## 💭궁금한 점 및 추가 정보

#### 1. hamiltonian vs second_q_op()
| 항목 | 설명 | 사용 상황 |
| ----------------------------------- | ------------------------------------------- | --------------------------- |
| `problem.hamiltonian.second_q_op()` | \*\*전자 해밀토니안(Ĥ\_elec)\*\*만을 다뤄 **정확한 해밀토니안** 생성 | - ElectronicEnergy 클래스 인스턴스 <br>- 해밀토니안만 필요할 때 (핵반발 제외)$E_{total} = E_{electric}+ E_{etc}$ 에서 $ E_{electric}$ <br> - 내부에 one-body, two-body 텐서(=정밀한 행렬 값들)가 들어 있음  <br>- problem.hamiltonian.second_q_op()은 이 데이터를 바탕으로 “수식적으로 정확한 해밀토니안 연산자” 를 생성 |
| `problem.second_q_ops()[0]` | **전자 해밀토니안 + 기타 속성 포함 가능**한 **통합 문제 정의**    | - ElectronicStructureProblem 클래스의 메서드 <br>- 알고리즘 실행에 자동으로 넘길 때  |


> second_q_op()의 출력에는 **핵반발 에너지(nuclear repulsion energy)**는 포함되지 않음.

전자 해밀토니안의 외부 상수항이기 때문에
계산 후 별도로 problem.hamiltonian.nuclear_repulsion_energy 에서 더해줘야함.


#### 2. second_q_ops 의 구조

second_q_ops = problem.second_q_ops()
| Index | 의미 |
| -- | -- |
| `[0]` | 해밀토니안 $\hat{H}_{elec}$ (전자 에너지)  |
| `[1]` | 전자 수 측정 연산자 (Particle Number)    |
| `[2]` | 각운동량 측정 연산자 (Angular Momentum)   |
| `[3]` | 자기화(Magnetization) 연산자           |
| `[4]` | 전기 쌍극자 모멘트 연산자 (Dipole Moment) 등 |

#### 3. 코드 각 요소의 의미 (화학/양자/하드웨어 관점)
| 변수 | 기능 설명 | 분자 해석 | 양자 하드웨어 관점 | 
| -- | -- | -- | -- | 
| `driver`       | PySCF 등을 통해 **분자의 구조 입력** 및 전자 적분 계산           | 분자의 좌표, 원자종, basis set    | 고전적 전처리 단계 (quantum 아님) | 
| `problem`      | 전자 구조 계산 문제 생성 | 전자 수, 오비탈 수, 상태 설정        | Qiskit Nature의 핵심 문제 정의 |        | |
| `second_q_ops` | 연산자들 생성 (`H`, `N`, `L`, `μ` 등) | 해밀토니안 등 연산자 구성            | 양자회로로 매핑할 수 있는 재료들      |  
| `main_op`      | 해밀토니안 연산자 추출 (`[0]`)                           | 전자 에너지 계산 대상              | 양자회로로 바꿔야 할 대상          |
| `mapper`       | Jordan-Wigner 등으로 **Fermionic → Qubit 연산자 변환** | second quantization 상태 유지 | 회로로 변환 가능한 형태로 매핑       |
| `qubit_op`     | 실제 큐비트 연산자 (Pauli Sum)                         | 에너지 계산할 연산자               | 실제 회로로 구현될 연산자          |
| `ansatz`       | 양자회로 구조 설계 (`TwoLocal`, `UCCSD`, 등)            | 상태 시도/모델링 방식              | 실제 회로 설계 (게이트 순서 포함)    |
| `optimizer`    | 클래식 최적화기 (COBYLA, SPSA 등)                      | 최적 상태 추적                  | 회로 파라미터 업데이트            |
| `estimator`    | 백엔드에서 expectation value 측정 $(⟨ψ\| H \| ψ⟩)$| 에너지 계산 | 실제 큐비트 측정값 수집 |
| `vqe_solver`   | 전체 실행기 (VQE 알고리즘 실행 객체)                        | 최저에너지 상태 찾기               | 회로 실행 + 결과 반환 제어기       |
| `result`       | VQE 최적 결과값 (eigenvalue)                        | 바닥 상태 에너지                 | 양자 하드웨어 최종 출력값          |
